In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [7]:
X=scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [40]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [11]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

C:\Users\A\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.5583 - loss: 0.6941 - val_accuracy: 0.5974 - val_loss: 0.6802
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6686 - loss: 0.6322 - val_accuracy: 0.6948 - val_loss: 0.6295
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7003 - loss: 0.6041 - val_accuracy: 0.7273 - val_loss: 0.5924
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7297 - loss: 0.5722 - val_accuracy: 0.7403 - val_loss: 0.5653
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7515 - loss: 0.5520 - val_accuracy: 0.7662 - val_loss: 0.5444
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7755 - loss: 0.5102 - val_accuracy: 0.7662 - val_loss: 0.5279
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7560 - loss: 0.5104 - val_accuracy: 0.7792 - val_loss: 0.5122
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7417 - loss: 0.5083 - val_accuracy: 0.7792 - 

In [13]:
# 1-> how to select appropriate optimizer
# 2-> no. of nodes in a layer
# 3-> how to select no. of layers
# 4-> all in all one model

# 1->

In [14]:
import keras_tuner as kt

In [15]:
def build_model(hp):
    
    model=Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer=hp.Choice('optimizer', values=['adam','sgd','rmsprop','adadelta'])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [16]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [17]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [19]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\A\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7567 - loss: 0.5248 - val_accuracy: 0.7727 - val_loss: 0.5137
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7511 - loss: 0.5195 - val_accuracy: 0.7662 - val_loss: 0.5039
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7859 - loss: 0.4814 - val_accuracy: 0.7662 - val_loss: 0.4970
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7738 - loss: 0.4993 - val_accuracy: 0.7727 - val_loss: 0.4909
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8063 - loss: 0.4639 - val_accuracy: 0.7727 - val_loss: 0.4864
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8073 - loss: 0.4548 - val_accuracy: 0.7792 - val_loss: 0.4821
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8085 - loss: 0.4419 - val_accuracy: 0.7792 - val_loss: 0.4788
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7690 - loss: 0.4861 - val_accuracy: 0.77

# 2->

In [22]:
def build_model(hp):

    model=Sequential()

    units=hp.Int('units', min_value=8, max_value=128)

    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [23]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5,
                     directory='mydir')

Reloading Tuner from mydir\untitled_project\tuner0.json


In [24]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [26]:
model=tuner.get_best_models(num_models=1)[0]

In [27]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.7820 - loss: 0.4794 - val_accuracy: 0.7922 - val_loss: 0.4703
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7633 - loss: 0.4856 - val_accuracy: 0.7987 - val_loss: 0.4643
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7848 - loss: 0.4577 - val_accuracy: 0.7987 - val_loss: 0.4599
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7729 - loss: 0.4501 - val_accuracy: 0.7857 - val_loss: 0.4580
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7733 - loss: 0.4683 - val_accuracy: 0.7987 - val_loss: 0.4566
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7865 - loss: 0.4562 - val_accuracy: 0.7987 - val_loss: 0.4549
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7906 - loss: 0.4353 - val_accuracy: 0.7922 - val_loss: 0.4543
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7977 - loss: 0.4225 - val_accuracy: 0.79

# 3->

In [28]:
def build_model(hp):

    model=Sequential()

    model.add(Dense(72, activation='relu', input_dim=8))

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

        model.add(Dense(72, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [29]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=3,
                     directory='mydir1')

Reloading Tuner from mydir1\untitled_project\tuner0.json


In [30]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [31]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8}

In [32]:
model=tuner.get_best_models(num_models=1)[0]

C:\Users\A\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [33]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.8033 - loss: 0.4658 - val_accuracy: 0.8052 - val_loss: 0.4740
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7872 - loss: 0.4452 - val_accuracy: 0.7403 - val_loss: 0.6120
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8008 - loss: 0.4214 - val_accuracy: 0.7597 - val_loss: 0.5121
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8023 - loss: 0.4307 - val_accuracy: 0.8247 - val_loss: 0.4776
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8201 - loss: 0.4081 - val_accuracy: 0.7727 - val_loss: 0.4958
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8104 - loss: 0.4165 - val_accuracy: 0.7922 - val_loss: 0.4821
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7872 - loss: 0.4246 - val_accuracy: 0.8182 - val_loss: 0.5096
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8321 - loss: 0.4040 - val_accuracy: 0.80

# 4->

In [41]:
def build_model(hp):

    model=Sequential()

    counter=0

    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

        if counter==0:

            model.add(
                Dense(
                    hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
                    input_dim=8
                    )
                )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:

            model.add(
                Dense(
                    hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
                    )
                )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

    return model

In [42]:
tuner=kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     directory="mydir2")

Reloading Tuner from mydir2\untitled_project\tuner0.json


In [43]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [44]:
model=tuner.get_best_models(num_models=1)[0]

In [45]:
model.fit(X_train, y_train, epochs=200, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7459 - loss: 0.4856 - val_accuracy: 0.7922 - val_loss: 0.4651
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7668 - loss: 0.4797 - val_accuracy: 0.7857 - val_loss: 0.4636
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7657 - loss: 0.4943 - val_accuracy: 0.7857 - val_loss: 0.4615
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7502 - loss: 0.4692 - val_accuracy: 0.7922 - val_loss: 0.4574
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7672 - loss: 0.4694 - val_accuracy: 0.7792 - val_loss: 0.4594
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7306 - loss: 0.5325 - val_accuracy: 0.7792 - val_loss: 0.4627
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7888 - loss: 0.4644 - val_accuracy: 0.7922 - val_loss: 0.4612
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7559 - loss: 0.4802 - val_accuracy: 0.76